In [1]:
import string
import numpy as np
import nltk
from sklearn.model_selection import train_test_split
import itertools
from operator import add
from scipy import sparse
import re
from collections import Counter

In [2]:
tagged_sentence = nltk.corpus.treebank.tagged_sents(tagset='universal')
tagged_words = list(set([tup for i in tagged_sentence for tup in i]))
vocab = list(set([word for word, tag in tagged_words]))
tags = list(set([tag for word, tag in tagged_words]))

In [5]:
train = tagged_sentence[:int(len(tagged_sentence)*0.7)]
test = tagged_sentence[int(len(tagged_sentence)*0.7+1):]

In [6]:
def feature_extractor(sentence, t_prev, t_pres, index):
    dic = {}
    word = sentence[index]
    dic[(word, t_pres)] = 1
    if index == 0:
        dic[('start', t_pres)] = 1
    else:
        dic[(t_prev, t_pres)] = 1
        dic[('pre_w '+sentence[index - 1], t_pres)] = 1
    if index <len(sentence)-1:
        dic[('aft_w '+sentence[index + 1], t_pres)] = 1
    else:
        dic[(t_pres, 'end')] = 1
    #if index >=2:
     #   dic[(tag[index -2]+' '+tag[index-1], t_pres)] = 1
    return dic

In [7]:
def feature_vector(sentence, tag):
    for j in range(len(sentence)):
        if j == 0:
            v = feature_extractor(sentence,'start',tag[j],j)
        else:
            v = Counter(v) +Counter(feature_extractor(sentence, tag[j-1], tag[j], j))
            v = dict(v)
    return v

In [8]:
def product(alpha, f):
    s = 0
    for key in f.keys():
        if key in alpha.keys():
            s = s+ alpha[key]*f[key]
        #else:
           # alpha[key] = 0
    return s

In [9]:
def Viterbi(alpha, sentence):
    n = len(sentence)
    t = [0]*n
    score = [0]*n
    state = [0]*n
    y = [0]*n
    curr_state_score = {}
    curr_state = {}
    last_state_score = {}
    
    for tag_ in tags:
        curr_state_score[tag_] = product(alpha, feature_extractor(sentence, 'start', tag_, 0))
        curr_state[tag_] = 'start'
    score[0] = curr_state_score.copy()
    state[0] = curr_state.copy()
    for i in range(1, n):
        last_state_score = curr_state_score.copy()
        for tag_c in tags:
            dic = {}
            for tag_l in tags:
                dic[tag_l] = last_state_score[tag_l]+product(alpha, feature_extractor(sentence, tag_l, tag_c, i))
            curr_state_score[tag_c] = max(dic.values())
            curr_state[tag_c] = max(dic, key=dic.get)
        score[i] = curr_state_score.copy()
        state[i] = curr_state.copy()
    y[-1] = max(curr_state_score, key = curr_state_score.get)
    for i in reversed(range(n-1)):
        curr_state = state[i+1]
        y[i] =  curr_state[y[i+1]]
    return y

In [34]:
def perceptron(training_set, T, avg = False):
    n = len(training_set)
    alpha = {}
    l = []
    for t in range(T):
        for i in range(n):
            tagged_s = training_set[i]
            tag = [tup[1] for tup in tagged_s ]
            sentence = [tup[0] for tup in tagged_s]
            z = Viterbi(alpha, sentence)
            #print (z)
            if z != tag:
                alpha = Counter(alpha) + Counter(feature_vector(sentence, tag)) - Counter(feature_vector(sentence, z))
                alpha = dict(alpha)
                if avg == True:
                    l.append(alpha)
    if avg == True:
        for al in l:
            alpha = Counter(alpha) + Counter(al)
        alpha = dict(alpha)
    return alpha

In [35]:
alpha = perceptron(train, 1, True)

In [37]:
alpha = {k: v /len(train) for k, v in alpha.items()}


In [38]:
sentence = [tup[0] for tup in test[4]]
tag = [tup[1] for tup in test[4]]

In [40]:
len(set(tag)-set(Viterbi(alpha, sentence)))

0

In [41]:
def err(test):
    count_w = 0
    dif = 0
    for tagged_s in test:
        sentence = [tup[0] for tup in tagged_s]
        tag = [tup[1] for tup in tagged_s]
        count_w  = count_w + len(sentence)
        dif = dif + len(set(tag)-set(Viterbi(alpha, sentence)))
    return dif/count_w

In [42]:
err(test)

0.010443070059819528

In [27]:
1- 0.0086

0.9914